# Analyse ambigous named entities

One specific token can have different labels in the context of named entity recognition. This experiment allows to read in all tokens and their labels and change the tokens' labels. E.g. if the token <code>Kiidjerwe</code> has the labels <code>LOC</code> and <code>ORG</code>, we may want to change all `ORG` labels to `LOC` if the token has the `ORG` label more than 95% of the times it's in the corpus.

In [ ]:
import os
import math

from estnltk import Text
from estnltk.converters import json_to_text
from estnltk.converters import text_to_json
from modules.results_extraction import extract_results, results_by_subdistribution

from estnltk.layer.layer import Layer
from estnltk import EnvelopingBaseSpan
from estnltk.layer_operations import flatten

from nervaluate import Evaluator
import pandas as pd

from modules.preprocessing_protocols import preprocess_text

Divided corpus file for filenames and layers to be removed later on:

In [ ]:
divided_corpus = os.path.join('..', 'data', 'divided_corpus.txt')
no_goldstandard_tags_location = os.path.join('..', 'data', 'files_without_goldstandard_annotations.txt')
testing_files_location = os.path.join('..', 'data', 'vallakohtufailid-json-flattened')

removed_layers = ['compound_tokens', 'tokens', 'words']

Read files from <code>path_to_read</code> and write files to <code>path_to_write</code>

In [ ]:
path_to_read = os.path.join('models',
                    'model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 
                    'model_gaz_loc_variants', 
                    'vallakohtufailid-trained-nertagger')

path_to_write = os.path.join('models',
                    'model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 
                    'model_gaz_loc_variants_me', 
                    'vallakohtufailid-trained-nertagger')

Get files without goldstandard annotations:

In [ ]:
with open(no_goldstandard_tags_location, 'r', encoding='UTF-8') as in_f:
    lines = in_f.readlines()

no_goldstandard_annotations = [line.strip() for line in lines]

In [ ]:
files = {}

with open(divided_corpus, 'r', encoding = 'UTF-8') as f:
    txt = f.readlines()

for filename in txt:
    file, subdistribution = filename.split(':')
    files[file] = subdistribution.strip()

In [ ]:
def multivalued_entity_dict(path, leave_out_O):
    me_dict = dict()
    texts_dict = dict()
    
    # Read all filenames from path
    for file in os.listdir(path):
        with open(os.path.join(path, file), 'r', encoding='UTF-8') as in_f:
            texts_dict[file] = json_to_text(in_f.read())
    
    # Create me_dict, which contains all named entities
    for file in texts_dict.keys():
        text = texts_dict[file]
        
        for entity in text.flat_wordner:
            named_entity = entity.text
            nertag = entity.nertag[0]
            labels = dict()
            if entity.text in me_dict.keys():
                if nertag in me_dict[named_entity]:
                    labels = me_dict[named_entity]
                    labels[nertag] = labels[nertag] + 1
                else:
                    labels = me_dict[named_entity]
                    labels[nertag] = 1
                    me_dict[named_entity] = labels
            else:
                me_dict[named_entity] = {nertag: 1}

    # If leave_out_O == True, leave out all named entites that
    # have other labels as well as the 'O' label 
    delete_keys = set()
    for key, value in me_dict.items():
        if len(value.items()) == 1:
            delete_keys.add(key)
        elif leave_out_O and 'O' in value.keys():
            delete_keys.add(key)

    for key in delete_keys:
        del me_dict[key]

    for key, value in me_dict.items():
        all_counts = 0
        for ne, count in value.items():
            all_counts += count

        for ne, count in value.items():
            percentage = round((count / all_counts) * 100, 3)
            value[ne] = percentage

    return me_dict

In [ ]:
def change_multivalued_entities_in_file(text, me_dict, threshold, threshold_O = 100):
    
    for i in range(len(text.flat_wordner)):
        entity = text.flat_wordner[i].text
        nertag = text.flat_wordner[i].nertag[0]
        
        # If the named entity is in the dict, check if it needs to be
        # changed due to the threshold
        
        if entity in me_dict:
            for key, value in me_dict[entity].items():
                before = text.flat_wordner[i].nertag[0]
                if key != 'O' and value > threshold and nertag != key:
                    text.flat_wordner[i].nertag = key
                    print(f'Muudatus sõnes {entity}: {before} -> {key}')
                elif key == 'O' and value > threshold_O and nertag != key:
                    text.flat_wordner[i].nertag = key
                    print(f'Muudatus sõnes {entity}: {before} -> {key}')
    return text

The `wordner` layer is converted to `ner` layer due to the results extraction being built upon `ner` tags and not `wordner` tags.

In [ ]:
def convert_wordner_to_ner(text):
    snt_labels = [label.nertag[0] for label in text.flat_wordner]
    
    text.pop_layer('flat_ner')
    text = preprocess_text(text)    
    nerlayer = Layer(name='ner', attributes=['nertag'], text_object=text, enveloping='words')
    
    entity_spans = []
    entity_type = None
    
    for span, label in zip(text.words, snt_labels):
        if entity_type is None:
            entity_type = label[2:]
        if label == "O":
            if entity_spans:
                nerlayer.add_annotation(EnvelopingBaseSpan(entity_spans),
                                        **{['nertag'][0]: entity_type})
                entity_spans = []
            continue
        if label[0] == "B" or entity_type != label[2:]:
            if entity_spans:
                nerlayer.add_annotation(EnvelopingBaseSpan(entity_spans),
                                        **{['nertag'][0]: entity_type})
                entity_spans = []
        entity_type = label[2:]
        entity_spans.append(span.base_span)
        
    text.add_layer(nerlayer)
    
    flat_ner = flatten(text['ner'], 'flat_ner')
    text.add_layer(flat_ner)
    
    for layer in removed_layers:
        text.pop_layer(layer)
    return text

---

In [ ]:
setups = [[True, 85], # Leave out O-s, if majority above 85% -> change
          [True, 90], # Leave out O-s, if majority above 90% -> change 
          [True, 95], # Leave out O-s, if majority above 95% -> change
          [False, 85, 95], # Don't leave out O-s, if majority of !O above 85% -> change, also change O if above 95%
          [False, 85, 99], # Don't leave out O-s, if majority of !O above 85% -> change, also change O if above 99%
          [False, 90, 95], # Don't leave out O-s, if majority of !O above 90% -> change, also change O if above 95%
          [False, 90, 99], # Don't leave out O-s, if majority of !O above 90% -> change, also change O if above 99%
          [False, 95, 95], # Don't leave out O-s, if majority of !O above 95% -> change, also change O if above 95%
          [False, 95, 99]] # Don't leave out O-s, if majority of !O above 95% -> change, also change O if above 99%

In [ ]:
all_results = dict()

for setup in setups:
    leave_out_O = setup[0]
    threshold = int(setup[1])
    try:
        threshold_O = setup[2]
    except:
        threshold_O = 100
        
    me_dict = multivalued_entity_dict(path_to_read, leave_out_O)
    
    for file in os.listdir(path_to_read):
        if file.endswith('json'):
            with open(os.path.join(path_to_read, file), 'r', encoding='UTF-8') as in_f:
                text = json_to_text(in_f.read())
            
            text = change_multivalued_entities_in_file(text, me_dict, threshold, threshold_O)
            text = convert_wordner_to_ner(text)
            text_to_json(text, file=os.path.join(path_to_write, file))
            
    all_results[f'{str(leave_out_O)}, {str(threshold)}, {str(threshold_O)}'] = (extract_results(files, no_goldstandard_annotations, path_to_write, testing_files_location, path_to_write))

---

In [ ]:
all_results

In [ ]:
df = dict()

for key in all_results.keys():
    results = results_by_subdistribution(all_results[key][0])
    precision = results[0]
    recall = results[1]
    f1 = results[2]
    df[key] = [precision, recall, f1]

In [ ]:
display(pd.DataFrame(df, index=['Precision', 'Recall', 'F1']))

The first `boolean` value states if `O` tags are completely left out or not.

The second value states the threshold for changing named entity labels from one to another:
e.g. if "Jaan" is 91% `B-PER` and 9% `I-PER` and if the threshold is `90`, all the `I-PER` tags
are also changed to `B-PER`. Should the threshold be for example 92, the named entity label 
stays unchanged.

The third value states the threshold for changing labels from or to `O` based on the threshold.